# Specreduce Workflow Exploration
Based heavily (initially) on the `pydis` machinery built by @jradavenport in 2015. All `pydis` pieces (or others as we swap-out, build-up, etc) will be moved flatly into this directory instead of importing for now.

Goal is to complete basic full reduction, illustrating a typical workflow, and then facilitate discussion about API and design choices for `specreduce` and it's interface with `ccdproc` and `specutils` (and others).

Also, look for opportunities to generalize what *observatories* or specific instruments could provide (e.g. trace templates as astropy models, reference cals as frist-guesses or fallbacks) to make reduction guides or even quick-look autoreductions.

**Put LOTS of comments and markdown discussion in!**

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})